In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [88]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [119]:
lp = StanProblem(bs_model);
D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  11%|███▌                           |  ETA: 0:00:01
  iterations:                                   895
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.02
  n_steps:                                      31
  is_accept:                                    true
  acceptance_rate:                              0.9763927659190847
  log_density:                                  -7.914204277695983
  hamiltonian_energy:                           16.06550245844311
  hamiltonian_energy_error:                     0.017868046272976557
  max_hamiltonian_energy_error:                 0.09941313307730226
  tree_depth:                                   5
  numerical_error:                              false
  step_size:                                    0.14914249053326567
  nom_step_size:                                0.14914249053326567
  is_adapt:                                     true



















Sampling  40%|████████████▎                  |  ETA: 0:00:00
  iterations:                                   3161
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9064916300019851
  log_density:                                  -5.680410764834427
  hamiltonian_energy:                           12.127811642625804
  hamiltonian_energy_error:                     -0.06584808204247139
  max_hamiltonian_energy_error:                 0.18131248311070536
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling  67%|████████████████████▊          |  ETA: 0:00:00
  iterations:                                   5354
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -3.125199584845127
  hamiltonian_energy:                           7.977536678133459
  hamiltonian_energy_error:                     -0.13366825911156255
  max_hamiltonian_energy_error:                 -0.13366825911156255
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling  87%|███████████████████████████▏   |  ETA: 0:00:00
  iterations:                                   6994
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9772925684495671
  log_density:                                  -5.848456318382831
  hamiltonian_energy:                           8.064871339961133
  hamiltonian_energy_error:                     0.04319032639595477
  max_hamiltonian_energy_error:                 -0.07950543492963558
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.4052342918202954
  log_density:                                  -8.868373854993063
  hamiltonian_energy:                           19.97580352442992
  hamiltonian_energy_error:                     -0.15188092584388002
  max_hamiltonian_energy_error:                 6.784952120901632
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.4043801560093249
  nom_step_size:                                0.4043801560093249
  is_adapt:                                     false
  mass_matrix:                    

┌ Info: Finished 8000 sampling steps for 1 chains in 0.497411802 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9964929543071683, 0.8576 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.404), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9772347795922866
│   average_acceptance_rate = 0.8720347766900625
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [130]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
        param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
    ]...);

In [138]:
hcat(eachcol(df)...)

10×10 Matrix{Vector{Float64}}:
 [10.6803, 6.45384, -2.24163, 2.46002, 5.62489, 7.50337, 7.816, 5.0193, 10.0454, 5.18654  …  10.8543, 6.86692, 0.799568, -6.00134, 3.91919, -0.457418, 14.0131, 10.4364, 7.83739, 2.67678]     …  [1.79395, 2.15365, 2.29508, 1.7082, 1.26627, 3.47871, 4.55486, 3.63693, 3.89038, 2.14736  …  4.47046, 0.556734, 1.05657, 4.23989, 0.541977, 0.945349, 1.70818, 5.26015, 3.93348, 2.46918]
 [5.70891, 10.3012, 4.24395, 11.0558, 9.81823, 8.31951, 5.38645, -4.35662, 9.83227, 7.12432  …  1.1718, 30.5928, 3.70927, 17.1807, -0.683814, 7.71773, 6.83029, 6.84319, 0.695842, 5.16589]       [7.63743, 2.74969, 3.66891, 6.35205, 0.518689, 1.80755, 2.94469, 2.34063, 6.40109, 2.40884  …  4.18128, 9.56908, 3.25226, 6.30062, 5.69385, 8.24627, 2.56901, 2.37046, 2.33592, 4.10594]
 [7.23747, -0.427832, 9.14782, 18.4352, 5.82655, 3.75256, 1.46422, 5.88634, 4.10779, -4.63588  …  0.863871, -4.51189, 1.40825, 7.37517, 10.208, 9.42749, 4.25254, 4.46672, 4.02189, 2.54327]      [0.632481, 6.085

In [134]:
mean(samples)

10-element Vector{Float64}:
  0.3173551467310577
  0.08433652958871951
 -0.0886958945091761
  0.06037996882064286
 -0.17066047954097616
 -0.06670302452397554
  0.37302740430830394
  0.059786873877665854
  4.384754324471194
  0.806020045873587